In [1]:
import torch
from transformers import AutoModel,AutoConfig,AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments,DataCollatorWithPadding
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import re
from pygtrans import Translate

In [2]:
# %%time
# # 加载数据
# excel_file = 'd:/基于深度学习的海量文本处理/第1阶段/10w.xlsx'
# data_frame = pd.read_excel(excel_file)

In [3]:
# # 随机抽样（预实验使用）
# data_frame = data_frame.sample(n=30)
# data_frame

In [4]:
# # 停用词预处理
# stop_words = ['您好','你好',':很高兴为您服务','请问有什么可以帮您','client','user',' ']
# sep_words = ['。', '!', '?', ',']
# def ProcessStopWords(text):
#     for word in stop_words:
#         text = text.replace(word,'')
#     text = text.replace(':','。').replace('。','',1) # 删除第一个。
#     # for word in sep_words:
#     #     text = text.replace(word, '[SEP]')
#     return text

# data_frame['转写文本'] = data_frame['转写文本'].map(ProcessStopWords)

In [5]:
# data_frame['转写文本'].iloc[0]

In [6]:
# # 标签预处理
# regex = re.compile(r'^.*?>(.*?)>.*?$')
# def ProcessLabels(text):
#     text = text.replace('>>','>').replace('10019','')
#     text = re.match(regex, text).groups()[0]
#     return text

# data_frame['服务请求'] = data_frame['服务请求'].map(ProcessLabels)

In [7]:
# 预览预处理结果
# data_frame['转写文本'].iloc[:10]

In [8]:
# # 训练复盘并分析数据集后，考虑在前面的处理把 样本数<1000 的剔除，即剔除下列：
# rm_labels = ['临时','其他','商机','资料信息','业务变更问题','投诉','故障']
# for rm_label in rm_labels:
#     data_frame.drop(data_frame[data_frame.服务请求 == rm_label].index, inplace=True)
# data_frame

In [9]:
# # 翻译转写文本
# client = Translate(target='en')
# temp = np.array([])

In [10]:
# # 测试
# pd.DataFrame(np.array([trans_res.translatedText for trans_res in client.translate(np.array(data_frame['转写文本'].iloc[0:2]).tolist())]))

In [11]:
# total_num = data_frame.shape[0]
# batch = 100
# epoch = int(total_num/batch)
# print(f'共{epoch}部分')
# for i in range(epoch):
#     if not i == epoch - 1:
#         print(f'{i}部分开始')
#         translated_res = [trans_res.translatedText for trans_res in client.translate(np.array(data_frame['转写文本'].iloc[i*batch:(i+1)*batch]).tolist())]
#     else:
#         print('最后一轮开始')
#         translated_res = [trans_res.translatedText for trans_res in client.translate(np.array(data_frame['转写文本'].iloc[i*batch:]).tolist())]
    
#     translated_res = np.array(translated_res)
#     temp =  np.concatenate((temp, translated_res), axis=0)
#     print(f'第{i}部分已完成')

In [12]:
# translated_res = temp
# len(translated_res)

In [13]:
# # 加入翻译
# # data_frame = data_frame.drop(columns=['translated_text'])
# data_frame.insert(0, 'translated_text', value=translated_res)

In [14]:
# # 处理翻译出错的符号
# def Finetune_translate(text):
#     text = text.replace('&#39;','\'')
#     return text
    
# data_frame['translated_text'] = data_frame['translated_text'].map(Finetune_translate)
# data_frame[:10]

In [15]:
# prompts = np.array(data_frame['translated_text'])
# choices = np.array(data_frame['服务请求'])

In [16]:
# data_frame['服务请求']

In [17]:
# # 取最大长度
# data_frame.insert(data_frame.shape[1], 'text_len',None)
# data_frame['text_len'] = data_frame['translated_text'].map(len)
# max_length_index = data_frame['text_len'].argmax()
# max_length = data_frame['text_len'].iloc[max_length_index]
# max_length_index, max_length

In [18]:
# # 观察文本长度分布（排除异常值）
# data_frame.boxplot('text_len', grid=False, showfliers=False, color='Black')
# plt.suptitle("")
# plt.xlabel("")
# plt.show()
# # 由图可知，取512足够覆盖正常样本
# max_length = 512

In [19]:
# # 去重choices，并保存原choices对应去重后的位置
# unique_choices = np.unique(choices)
# labels = np.array([np.argwhere(unique_choices==v)[0]  for v in choices])
# unique_choices.shape, labels.shape

In [20]:
# # 加入标签
# data_frame.insert(0, 'label', value=labels)
# data_frame[:10]

In [21]:
# # 取出特征与labels
# df = data_frame[['label', 'translated_text', '服务请求']]
# df[:10]
# # 统计
# df['label'].value_counts(ascending=True).plot.barh()
# plt.title("Frequency of Classes")
# plt.show()

In [22]:
# # 手动处理样本非均衡情况
# df4 = df[df['label'] == 4].sample(n=30000)
# df2 = df[df['label'] == 2]
# df1 = pd.DataFrame(np.repeat(df[df['label'] == 1].values, 2, axis=0), columns=df.columns)
# df0 = pd.DataFrame(np.repeat(df[df['label'] == 0].values, 2, axis=0), columns=df.columns)
# df3 = pd.DataFrame(np.repeat(df[df['label'] == 3].values, 2, axis=0), columns=df.columns)

# df = pd.concat([df0, df1, df2, df3, df4], ignore_index=True)

In [23]:
# # 先排序 label，以便后续充分打乱
# df = df.sort_values(by='服务请求')

In [24]:
save_excel_path = './PreProcess.xlsx'

In [25]:
# df.to_excel(save_excel_path) # 保存翻译结果，方便重复使用
# df

In [52]:
# 用于直接读取预处理完成的结果
df = pd.read_excel(save_excel_path)
df = df.drop(columns='Unnamed: 0')
df.dropna(inplace=True) # drop 空行，防止 tokenize 失败

# 预处理结果相关参数
max_length = 512
classic_num = 5

# 预览
print(df.shape[0])
df[:10]

99326


,label,translated_text,服务请求
0,0,"Hey, that is my account. The current network i...",不满
7637,0,"How can I help you. Hey, let's see if my numbe...",不满
2,0,Have the issues I reported not been resolved y...,不满
3,0,". Sorry, my number is blocked. Sir, please hol...",不满
4,0,I would like to ask if there is anything I can...,不满
5,0,"Why. Um. I said, hey, my TV is out of order. I...",不满
7642,0,Can this card be used? Why can't it be activat...,不满
7643,0,This is my card and I want to cancel it. Is it...,不满
7644,0,Hey that one. The broadband you have at your h...,不满
9,0,Then help me restart it. Restart what? Sir. Ah...,不满


In [53]:
# 创建数据集
ds = DatasetDict({'train': Dataset.from_pandas(df)})
# ds['train'] = ds['train'].rename_column('转写文本','text')
ds = ds.remove_columns(['服务请求'])

ds['train'] = ds['train'].rename_columns({'translated_text':'text'})
ds = ds['train'].train_test_split(0.2, shuffle=True) # 按 8:2 分割数据集

# ds = ds.shuffle(88) # 再次打乱数据集
ds, ds['train'][0], ds['test'][0]

(DatasetDict({
     train: Dataset({
         features: ['label', 'text', '__index_level_0__'],
         num_rows: 79460
     })
     test: Dataset({
         features: ['label', 'text', '__index_level_0__'],
         num_rows: 19866
     })
 }),
 {'label': 2,
  'text': 'Hey, why is my number suspended? I thought it was suspended and then I just recharged 50 yuan and I said it was suspended. Um. Well, now I can help you see that the current number is experiencing abnormal behavior, so this number has been suspended. I will send you a text message later. There is a link in the text message. You can click on the text message later. Just follow the link to perform a self-service reconnection and you can use it normally. Why did you occupy it? Well, it was detected by the system that there was abnormal behavior. Then I will send you the bonus method later. You can use it normally after completing the self-service reconnection according to the text message steps. OK, all right. So what othe

In [48]:
%%time
model_path = './bert-base-cased/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=max_length)

tokenized_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/79460 [00:00<?, ? examples/s]

Map:   0%|          | 0/19866 [00:00<?, ? examples/s]

Wall time: 49 s


In [29]:
# 模型
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # 允许不同长度tensor的存在
model_config = AutoConfig.from_pretrained(model_path)
model_config.num_labels = classic_num
model = AutoModelForSequenceClassification.from_config(model_config)

In [30]:
tokenized_ds['train'][0]

NameError: name 'tokenized_ds' is not defined

In [ ]:
# 评估
accuracy = evaluate.load('./evaluate/metrics/accuracy/accuracy.py')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# Adam 训练阶段
training_args = TrainingArguments('./output',evaluation_strategy='epoch',save_strategy='epoch', learning_rate=5e-7, 
                                    load_best_model_at_end=True, num_train_epochs=3, use_cpu=True)
trainer = Trainer(model, args=training_args, train_dataset=tokenized_ds['train'], eval_dataset=tokenized_ds['test'], 
                  tokenizer=tokenizer, data_collator=data_collator, compute_metrics=compute_metrics)

In [ ]:
%%time
# 训练
trainer.train(resume_from_checkpoint=False)

In [ ]:
# SGD 训练阶段
ds = ds.shuffle(88) # 再次打乱数据集
optimizer = torch.optim.SGD(model.parameters(), lr=5e-8)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(epoch+1))
training_args = TrainingArguments('./output',evaluation_strategy='epoch',save_strategy='epoch',
                                load_best_model_at_end=True, num_train_epochs=5)
trainer = Trainer(model, args=training_args, train_dataset=tokenized_ds['train'], eval_dataset=tokenized_ds['test'], 
                  tokenizer=tokenizer, data_collator=data_collator, 
                  compute_metrics=compute_metrics, optimizers=(optimizer, lr_scheduler))

In [ ]:
%%time
# 精准训练
trainer.train(resume_from_checkpoint=False)